#### This notebook tests & runs the pipeline for extracting the depths at lineage defining SNP sites for sequenced samples (which are then used to evaluate the minor allele frequency and determine if sample is *mixed* via F2 metric)

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

############################################################################################################################################################################################################################

### Prepare script that extracts the depths at lineage defining SNP sites from VCF files

############################################################################################################################################################################################################################

#### Inputs for Script

In [6]:
parent_output_dir = sys.argv[1] #INPUT 1 - directory where VCF file for sequenced isolated is stored & filename (ex. '/n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/')
tag = sys.argv[2] #INPUT 2 - sample isolate tag (ex. '02-R1447')

#### Import Packages


In [7]:
import vcf
import os
import pandas as pd
import numpy as np
from itertools import compress
import ast
import itertools
import time
import sys
import pickle
import shutil

import Bio
from Bio.Alphabet import IUPAC
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import pairwise2
from Bio import SeqIO
from Bio.Graphics import GenomeDiagram
from Bio.SeqUtils import GC
from Bio.Align.Applications import MuscleCommandline
from StringIO import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import MutableSeq

#### Lineage Defining SNPs

Import Lineage Defining SNP sets from **Coll et. al. 2014**

In [42]:
lineage_defining_SNPs = pd.read_csv('/home/rv76/Farhat_Lab/CSV_files/Coll2014_LinSpeSNPs_final.csv')

In [43]:
lineage_defining_SNPs.head()

,lineage,position,gene_coordinate,allele_change,codon_number,codon_change,amino_acid_change,locus_Id,gene_name,mutation_type,essential,diagnostic_snp
0,lineage1,6112,873,G/C,291,ATG/ATC,M/I,Rv0005,gyrB,nonsyn,essential,no
1,lineage1,8452,1151,C/T,384,GCA/GTA,A/V,Rv0006,gyrA,nonsyn,nonessential,no
2,lineage1,13298,261,G/C,87,ATC/ATG,I/M,Rv0010c,0,nonsyn,nonessential,no
3,lineage1,22961,221,T/G,74,GAT/GCT,D/A,Rv0018c,pstP,nonsyn,nonessential,no
4,lineage1,26347,535,C/G,179,GAC/CAC,D/H,Rv0021c,0,nonsyn,nonessential,no


In [44]:
np.shape(lineage_defining_SNPs)

(6915, 12)

In [47]:
SNP_per_branch_count = pd.Series( Counter( list( lineage_defining_SNPs.lineage ) ) ).sort_values(ascending = True)

In [48]:
SNP_per_branch_count.head(n=10)

lineage3.1           1
lineage1.2           5
lineage4.7          11
lineage3.1.2        14
lineage4.6          16
lineage4.1.2        17
lineage4.3.4.2.1    18
lineage3.1.2.1      24
lineage2.2.1.1      25
lineage4.8          25
dtype: int64

According to **Wyllie et. al. 2018**, drop SNP sets corresponding to branches with fewer than 20 SNPs (we are left with 57 branches , not 55 as reported in Wyllie et. al.)

In [49]:
excluded_branches = ['lineage1.2' , 'lineage3.1' , 'lineage3.1.2' , 'lineage4.1.2' , 'lineage4.3.4.2.1' , 'lineage4.6' , 'lineage4.7']

#create a filter to filter out any SNP corresponding to the excluded lineages
included_branch_filter = [lineage not in excluded_branches for lineage in lineage_defining_SNPs.lineage]
lineage_defining_SNPs = lineage_defining_SNPs[included_branch_filter]
lineage_defining_SNPs.reset_index(inplace = True)

In [50]:
np.shape(lineage_defining_SNPs)

(6833, 13)

List of branches that we are analyzing

In [56]:
set( lineage_defining_SNPs.lineage )

{'lineage1',
 'lineage1.1',
 'lineage1.1.1',
 'lineage1.1.1.1',
 'lineage1.1.2',
 'lineage1.1.3',
 'lineage1.2.1',
 'lineage1.2.2',
 'lineage2',
 'lineage2.1',
 'lineage2.2',
 'lineage2.2.1',
 'lineage2.2.1.1',
 'lineage2.2.1.2',
 'lineage2.2.2',
 'lineage3',
 'lineage3.1.1',
 'lineage3.1.2.1',
 'lineage3.1.2.2',
 'lineage4',
 'lineage4.1',
 'lineage4.1.1',
 'lineage4.1.1.1',
 'lineage4.1.1.2',
 'lineage4.1.1.3',
 'lineage4.1.2.1',
 'lineage4.2',
 'lineage4.2.1',
 'lineage4.2.2',
 'lineage4.2.2.1',
 'lineage4.3',
 'lineage4.3.1',
 'lineage4.3.2',
 'lineage4.3.2.1',
 'lineage4.3.3',
 'lineage4.3.4',
 'lineage4.3.4.1',
 'lineage4.3.4.2',
 'lineage4.4',
 'lineage4.4.1',
 'lineage4.4.1.1',
 'lineage4.4.1.2',
 'lineage4.4.2',
 'lineage4.5',
 'lineage4.6.1',
 'lineage4.6.1.1',
 'lineage4.6.1.2',
 'lineage4.6.2',
 'lineage4.6.2.1',
 'lineage4.6.2.2',
 'lineage4.8',
 'lineage4.9',
 'lineage5',
 'lineage6',
 'lineage7',
 'lineageBOV',
 'lineageBOV_AFRI'}

In [13]:
def check_call_is_SNP(ref_allele , alt_alleles):
    
    '''This function checks to see if Call is a SNP and not an InDel or Structural Variant'''
    
    #check that Reference Allele is 1 base
    if len(ref_allele) == 1:
        
        #check to see if there is no alternate allele
        if alt_alleles == [None]:
            
            good_SNP = True
                
        #if there is an alternate allele(s) check to see that they are all just 1 base
        elif ( sum( [(len(alt_allele) == 1) for alt_allele in alt_alleles] ) == len(alt_alleles) ):
            
            good_SNP = True
            
        #at least 1 alternate allele was longer than 1 base
        else:
            
            good_SNP = False
    
    #reference allele was longer than 1 base        
    else:
        good_SNP = False
            
    return good_SNP

In [28]:
def get_lineage_defining_SNP_depths(tag , parent_output_dir):

    '''
    This function takes as input, the isolate tag, then extracts SINGLE base calls from
    the VCF file generated by Pilon specified as a lineage defining SNP from Coll et. al. 2014. 
    The function returns a DataFrame for a single isolate that contains base calling information
    for all of these SNP positions.
    '''

    #path to VCF file
    VCF_file = parent_output_dir + tag + '/pilon/' + tag + '.vcf'

    vcf_reader = vcf.Reader(open(VCF_file , 'r'))

    #create dictionaries to store information for each call
    lineage_dict = {}
    position_dict = {}
    depth_dict = {}
    max_base_count_dict = {}

    #indexer for dataframe containing lineage defining SNPs
    index = 0

    #iterate through each record from VCF file
    for record in vcf_reader:

        #check to see that the record does not correspond to a variant call that is a Structural Variant (SVTYPE)
        if 'SVTYPE' not in record.INFO.keys():

            #position of variant on the Reference Genome
            reference_position = record.POS 

            #reference allele
            ref_allele = record.REF

            #alternate alleles (if there are any)
            alt_alleles = record.ALT

            #check to see if call is at a lineage defining site AND that call is a SNP
            if ( reference_position in list( lineage_defining_SNPs.position ) ) and check_call_is_SNP(ref_allele , alt_alleles):

                ######## Retrieve Relevant information for filtering quality of Base Call ########
                # Mean Base Quality @ locus
                BQ = record.INFO['BQ']

                # Mean Mapping Quality @ locus
                MQ = record.INFO['MQ']

                # Number of Reads w/ Deletion 
                DC = record.INFO['DC']

                # Number of Reads w/ Insertion
                IC = record.INFO['IC']  
                
                # Depth of Valid Reads in Pileup
                VD = record.INFO['DP']

                ### Filtering Criteria
                #---> Mean Base Quality > 20
                #---> Mean Mapping Quality > 30
                #---> No Reads Supporting Insertions
                #---> No Reads Supporting Deletions
                #---> Number of High Quality Reads >= 25
                if (BQ > 20) and (MQ > 30) and (DC == 0) and (IC == 0) and (VD >= 25): #SNP passed filtering criteria!

                    # Valid read depth; some reads may have been filtered
                    total_depth = record.INFO['DP']

                    #Count of As, Cs, Gs, Ts at locus
                    base_counts = record.INFO['BC']

                    #most common base - depth
                    most_common_base_depth = np.max(base_counts)

                    #lineage that is defined by SNP
                    lineage_defined = list( lineage_defining_SNPs[lineage_defining_SNPs.position == reference_position].lineage )[0]

                    #After filtering for high-quality lineage-defining calls, store all of the pertinent information about the Base Call
                    lineage_dict[index] = lineage_defined
                    position_dict[index] = reference_position
                    depth_dict[index] = total_depth
                    max_base_count_dict[index] = most_common_base_depth

                    index += 1

    #convert dictionaries to series
    lineage = pd.Series(lineage_dict)
    position = pd.Series(position_dict)
    depth = pd.Series(depth_dict)
    max_base_count = pd.Series(max_base_count_dict)

    #create DataFrame to hold all lineage defining SNPs
    lineage_SNP_depths_from_sample_DF = pd.DataFrame()
    lineage_SNP_depths_from_sample_DF['lineage'] = lineage
    lineage_SNP_depths_from_sample_DF['position'] = position
    lineage_SNP_depths_from_sample_DF['depth'] = depth
    lineage_SNP_depths_from_sample_DF['max_base_count'] = max_base_count

    #calculate the minor depth for each SNP site
    lineage_SNP_depths_from_sample_DF['minor_depth'] = lineage_SNP_depths_from_sample_DF['depth'] - lineage_SNP_depths_from_sample_DF['max_base_count']

    return lineage_SNP_depths_from_sample_DF

In [ ]:
lineage_SNP_depths_from_sample_DF = get_lineage_defining_SNP_depths(tag , parent_output_dir)

#pickle DataFrame that has total depth & minor depth for all lineage defining SNPs for this isolate
lineage_SNP_depths_from_sample_DF.to_pickle(parent_output_dir + tag + '/' + tag + '_lineage_defining_SNP_depths.pkl')

In [ ]:
lineage_SNP_depths_from_sample_DF.head()

In [ ]:
np.shape(lineage_SNP_depths_from_sample_DF)

############################################################################################################################################################################################################################

### Submit Jobs - Longitudinal Samples

############################################################################################################################################################################################################################

In [6]:
import os
import pandas as pd
from slurmpy import Slurm
import numpy as np

#### Import sample annotation for Longitudinal Samples

In [7]:
sample_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/CSV_files/sample_annotation_files/cetr_casali_walker_trauner_witney_xu_guerra_bryant_fastq_path_names_and_JankyPipe_tags.csv' , sep  = ',').set_index('patient_id')

In [8]:
np.shape(sample_annotation)

(614, 7)

In [9]:
sample_annotation.head()

,Unnamed: 0,fastq_files,population,run_ID,sample_ID,sample_order,tag
patient_id,,,,,,,
I0005973-8,0,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MQFF00000000,Peru3062,1,Peru3062
I0005973-8,1,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MQKH00000000,Peru3315,2,Peru3315
I0005229-5,2,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MQAO00000000,Peru2908,1,Peru2908
I0005229-5,3,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MQKD00000000,Peru3278,2,Peru3278
I0005235-2,4,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,CETR,MQBA00000000,Peru2921,1,Peru2921


In [10]:
all_sample_tags = list( set( list(sample_annotation.tag) ) )

#directory where VCF file for sequenced isolated is stored & filename
parent_output_dir = '/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/output/'

In [ ]:
for sample_tag in all_sample_tags:
    
    lineage_defining_SNPs_job = 'python /n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/python_scripts/lineage_defining_SNP_depths_collection.py ' + parent_output_dir + ' ' + sample_tag

    #directory where you want output + error files
    os.chdir('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/SLURM_logs/lineage_defining_SNP_depth_SLURM_logs/')

    job_name = sample_tag

    s = Slurm(job_name , {'partition':'short' , 'n':'1' , 't':'0-03:00:00' , 'mem-per-cpu':'24G' , 'mail-type':'FAIL' , 'mail-user':'roger_vargas@g.harvard.edu'})

    #submits the job
    job_id = s.run(lineage_defining_SNPs_job)

    print job_name  + ' : ' +  str(lineage_defining_SNPs_job)

############################################################################################################################################################################################################################

### Submit Jobs - Replicate Samples

############################################################################################################################################################################################################################

In [3]:
import os
import pandas as pd
from slurmpy import Slurm
import numpy as np

#### Import sample annotation for Replicate Samples

In [4]:
sample_annotation = pd.read_csv('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/CSV_files/sample_annotation_files/REPLICATE_fastq_path_names_and_JankyPipe_tags.csv' , sep  = ',').set_index('patient_id')

#drop duplicate isolates (may have arisen from Trauner fudging of annotation file)
drop_duplicate_isolate_filter = [not redundant for redundant in list( sample_annotation.duplicated('tag') ) ]
sample_annotation = sample_annotation[drop_duplicate_isolate_filter]

In [5]:
sample_annotation.head()

,Unnamed: 0,fastq_files,population,run_ID,sample_ID,sample_order,tag,successful_run
patient_id,,,,,,,,
I0002918-6,0,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,LC_REP,NaN,Peru4092,0,Peru4092,yes
I0002918-6,1,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,LC_REP,MMMI00000000,Peru3380,0,Peru3380,yes
I0003710-6,2,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,LC_REP,NaN,Peru2905,0,Peru2905,yes
I0003710-6,3,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,LC_REP,MMBH00000000,Peru4104,0,Peru4104,yes
I0003922-7,4,/n/data1/hms/dbmi/farhat/cetr_strains/good_wgs...,LC_REP,NaN,Peru3016,0,Peru3016,yes


In [4]:
all_sample_tags = list( set( list(sample_annotation.tag) ) )

#directory where VCF file for sequenced isolated is stored & filename
parent_output_dir = '/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/JankyPipe/output_REPLICATES/'

In [5]:
for sample_tag in all_sample_tags:
    
    lineage_defining_SNPs_job = 'python /n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/python_scripts/lineage_defining_SNP_depths_collection.py ' + parent_output_dir + ' ' + sample_tag

    #directory where you want output + error files
    os.chdir('/n/data1/hms/dbmi/farhat/Roger/inhost_TB_dynamics_project/SLURM_logs/lineage_defining_SNP_depth_SLURM_logs/')

    job_name = sample_tag

    s = Slurm(job_name , {'partition':'short' , 'n':'1' , 't':'0-03:00:00' , 'mem-per-cpu':'24G' , 'mail-type':'FAIL' , 'mail-user':'roger_vargas@g.harvard.edu'})

    #submits the job
    job_id = s.run(lineage_defining_SNPs_job)

    print job_name  + ' : ' +  str(lineage_defining_SNPs_job)

submitted: Submitted batch job 32096028


Peru4948 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4948


submitted: Submitted batch job 32096029
submitted: Submitted batch job 32096030
submitted: Submitted batch job 32096031
submitted: Submitted batch job 32096032
submitted: Submitted batch job 32096033


Peru4679 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4679
02-R0848 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ 02-R0848
Peru4675 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4675
Peru4671 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4671
Peru4670 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4670


submitted: Submitted batch job 32096034
submitted: Submitted batch job 32096035
submitted: Submitted batch job 32096036
submitted: Submitted batch job 32096037
submitted: Submitted batch job 32096038


...

submitted: Submitted batch job 32096174
submitted: Submitted batch job 32096175
submitted: Submitted batch job 32096176
submitted: Submitted batch job 32096177
submitted: Submitted batch job 32096178


Peru4700 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4700
Peru4703 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4703
Peru4702 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4702
01-R0737 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ 01-R0737
Peru4707 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4707


submitted: Submitted batch job 32096179
submitted: Submitted batch job 32096180
submitted: Submitted batch job 32096181
submitted: Submitted batch job 32096182
submitted: Submitted batch job 32096183


Peru4706 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4706
Peru4709 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4709
Peru4977 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4977
Peru4974 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4974
Peru4972 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/Roger/JankyPipe/output_REPLICATES/ Peru4972
04-R0266 : python /home/rv76/Farhat_Lab/Python_Scripts/Final_5/lineage_defining_SNP_depths_collection.py /n/data1/hms/dbmi/farhat/

submitted: Submitted batch job 32096184
submitted: Submitted batch job 32096185
submitted: Submitted batch job 32096186
submitted: Submitted batch job 32096187
submitted: Submitted batch job 32096188
